In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import cv2
import imutils
import numpy as np
import matplotlib.pyplot as plt
import time
from os import listdir
from tensorflow.keras.preprocessing import image

%matplotlib inline

In [2]:
def crop_brain_contour(image, plot=False):
    
    #import imutils
    #import cv2
    #from matplotlib import pyplot as plt
    
    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)
    

    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])
    
    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]            

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)
        
        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)
        
        plt.title('Original Image')
            
        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both', 
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')
        
        plt.show()
    
    return new_image

In [3]:
image_width, image_height = (240, 240)
image = cv2.imread("Y162.jpg")

In [4]:
# crop the brain and ignore the unnecessary rest part of the image
image = crop_brain_contour(image, plot=False)
# resize image
image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
# normalize values
image = image / 255.

In [5]:
image.shape

(240, 240, 3)

In [6]:
image = image.reshape(1,240,240,3)
image.shape

(1, 240, 240, 3)

In [7]:
best_model = load_model(filepath='models/cnn-parameters-improvement-23-0.91.model')

In [8]:
output = best_model.predict(image)

In [9]:
output[0][0]

0.9997327

In [10]:
if output[0][0] < 0.5:
    print("Non-Tumorous")
else:
    print("Tumorous")

Tumorous


In [14]:
import pandas as pd
import os
yes_result = pd.DataFrame(columns=["Filename","actual","Predicted"])
yes_list = os.listdir("yes")

yes_result["Filename"] = yes_list
yes_result["actual"] = "Tumorous"

In [15]:
for i in range(len(yes_result["Filename"])):
    
    path = os.path.abspath(os.getcwd()) + "\\yes\\{}".format(yes_result["Filename"][i])
    image_width, image_height = (240, 240)
    image = cv2.imread(path)
    image = crop_brain_contour(image, plot=False)
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    image = image / 255.
    image = image.reshape(1,240,240,3)
    
    best_model = load_model(filepath='models/cnn-parameters-improvement-23-0.91.model')

    output = best_model.predict(image)
        
    if output[0][0] < 0.5:
        yes_result["Predicted"][i] = "Non-Tumorous"
    else:
        yes_result["Predicted"][i] = "Tumorous"

In [16]:
yes_result

,Filename,actual,Predicted
0,Y1.jpg,Tumorous,Tumorous
1,Y10.jpg,Tumorous,Tumorous
2,Y100.JPG,Tumorous,Non-Tumorous
3,Y101.jpg,Tumorous,Tumorous
4,Y102.jpg,Tumorous,Tumorous
...,...,...,...
150,Y95.jpg,Tumorous,Tumorous
151,Y96.jpg,Tumorous,Tumorous
152,Y97.JPG,Tumorous,Tumorous
153,Y98.JPG,Tumorous,Tumorous


In [17]:
no_result = pd.DataFrame(columns=["Filename","actual","Predicted"])
no_list = os.listdir("no")

no_result["Filename"] = no_list
no_result["actual"] = "Non-Tumorous"

In [18]:
for i in range(len(no_result["Filename"])):
    
    path = os.path.abspath(os.getcwd()) + "\\no\\{}".format(no_result["Filename"][i])
    image_width, image_height = (240, 240)
    image = cv2.imread(path)
    image = crop_brain_contour(image, plot=False)
    image = cv2.resize(image, dsize=(image_width, image_height), interpolation=cv2.INTER_CUBIC)
    image = image / 255.
    image = image.reshape(1,240,240,3)
    
    best_model = load_model(filepath='models/cnn-parameters-improvement-23-0.91.model')

    output = best_model.predict(image)
        
    if output[0][0] < 0.5:
        no_result["Predicted"][i] = "Non-Tumorous"
    else:
        no_result["Predicted"][i] = "Tumorous"

In [19]:
no_result

,Filename,actual,Predicted
0,1 no.jpeg,Non-Tumorous,Non-Tumorous
1,10 no.jpg,Non-Tumorous,Non-Tumorous
2,11 no.jpg,Non-Tumorous,Non-Tumorous
3,12 no.jpg,Non-Tumorous,Non-Tumorous
4,13 no.jpg,Non-Tumorous,Non-Tumorous
...,...,...,...
93,No18.jpg,Non-Tumorous,Non-Tumorous
94,No19.jpg,Non-Tumorous,Non-Tumorous
95,No20.jpg,Non-Tumorous,Non-Tumorous
96,No21.jpg,Non-Tumorous,Non-Tumorous


In [20]:
result = pd.concat([yes_result,no_result])
result

,Filename,actual,Predicted
0,Y1.jpg,Tumorous,Tumorous
1,Y10.jpg,Tumorous,Tumorous
2,Y100.JPG,Tumorous,Non-Tumorous
3,Y101.jpg,Tumorous,Tumorous
4,Y102.jpg,Tumorous,Tumorous
...,...,...,...
93,No18.jpg,Non-Tumorous,Non-Tumorous
94,No19.jpg,Non-Tumorous,Non-Tumorous
95,No20.jpg,Non-Tumorous,Non-Tumorous
96,No21.jpg,Non-Tumorous,Non-Tumorous


In [21]:
result.drop(["Filename"],axis=1,inplace = True)
result

,actual,Predicted
0,Tumorous,Tumorous
1,Tumorous,Tumorous
2,Tumorous,Non-Tumorous
3,Tumorous,Tumorous
4,Tumorous,Tumorous
...,...,...
93,Non-Tumorous,Non-Tumorous
94,Non-Tumorous,Non-Tumorous
95,Non-Tumorous,Non-Tumorous
96,Non-Tumorous,Non-Tumorous


In [22]:
pd.crosstab(result["actual"],result["Predicted"])

Predicted,Non-Tumorous,Tumorous
actual,,
Non-Tumorous,95,3
Tumorous,26,129
